## 행위 패턴 개요

행위 패턴은 이름 그대로 객체의 역할에 초점을 둔다. 더 큰 기능을 구현하기 위한 객체 간의 상호 작용을 중요시한다. 행동 패턴에서 객체는 상호작용하지만 느슨하게 결합돼 있다. 

## 옵서버 패턴 이해

옵서버 패턴에서 객체(서브젝트)는 자식(옵서버)의 목록을 유지하며 서브젝트가 옵서버에 정의된 메소드를 호출할 때마다 옵서버에 이를 알린다. 분산형 애플리케이션에는 사용자가 요청한 작업을 수행하는 다수의 서비스가 엮여있다. 각 서비스가 수행하는 다양한 연산은 객체 상태에 직접적인 영향을 받는다.

브로드캐스트나 게시/구독(pub/sub) 시스템에서 옵서버 디자인 패턴이 자주 사용된다. 블로그를 예로 들면, 특정 블로그를 구독하고 새로운 글이 추가됐을 때 알람을 받는 경우가 있을 때 이 상황을 옵서버 패턴으로 구현한다면 블로그는 구독자 또는 옵서버의 목록을 유지하는 서브젝트다. 블로그에 새로운 글이 등록되면 이메일 또는 각 옵서버가 선택한 방식으로 알림을 보낸다.

옵서버 패턴의 목적은 다음과 같다.

* 객체 간 일대다 관계를 형성하고 객체의 상태를 다른 종속 객체에 자동으로 알린다.
* 서브젝트의 핵심 부분을 캡슐화한다.

옵서버 패턴은 다음과 같은 상황에 적합하다.

* 분산 시스템의 이벤트 서비스를 구현할 때
* 뉴스 에이전시 프레임워크
* 주식 시장 모델

파이썬으로 구현한 옵서버 디자인 패턴

In [1]:
class Subject:
    def __init__(self):
        self.__observers = []
    
    def register(self, observer):
        self.__observers.append(observer)
        
    def notifyAll(self, *args, **kwargs):
        for observer in self.__observers:
            observer.notify(self, *args, **kwargs)

class Observer1:
    def __init__(self, subject):
        subject.register(self)
    
    def notify(self, subject, *args):
        print(type(self).__name__,':: Got', args, 'From', subject)


class Observer2:
    def __init__(self, subject):
        subject.register(self)
    
    def notify(self, subject, *args):
        print(type(self).__name__,':: Got', args, 'From', subject)

subject = Subject()
observer1 = Observer1(subject)
observer2 = Observer2(subject)
subject.notifyAll('notification')

Observer1 :: Got ('notification',) From <__main__.Subject object at 0x10544ec18>
Observer2 :: Got ('notification',) From <__main__.Subject object at 0x10544ec18>
